# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_pd = pd.read_csv ("../WeatherPy/output_data/cities.csv")
cities_pd.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,2180815,Tuatapere,100,NZ,1665318059,74,-46.1333,167.6833,12.32,4.34
1,3347019,Mossamedes,0,AO,1665318059,69,-15.1961,12.1522,26.35,5.14
2,5880054,Barrow,0,US,1665318059,93,71.2906,-156.7887,-6.99,3.60
3,5282297,Bethel,0,US,1665318060,86,41.3712,-73.4140,7.49,0.45
4,5380437,Pacific Grove,100,US,1665318060,94,36.6177,-121.9166,13.90,2.24


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configuring Gmaps
gmaps.configure(api_key=g_key)

# Getting he information from the Data Frame to be plotted in the Heatmap
# Lat and Lng as locations and Humidity as the weight
locations = cities_pd[["Lat","Lng"]]
weights = cities_pd["Humidity"]

# Configuring the Gmaps figure and Heatmap layer
fig = gmaps.figure(center=(20.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=weights, max_intensity=100)

# Add Heatmap layer to map.
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
 - Max Temp between 27C and 33C.
 - Wind speed is lower than 20 km/h.
 - Zero cloudiness
* Drop any rows will null values.

In [4]:
# Identifying the group of cities for vacation, based on the criteria above
vacation_df = cities_pd.loc[(cities_pd["Max Temp"] < 33) & (cities_pd["Max Temp"] > 27) & \
                            (cities_pd["Wind Speed"] < 20) & (cities_pd["Cloudiness"] == 0)]

# Dropping rows with null values
vacation_df = vacation_df.dropna()

vacation_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
94,5855927,Hilo,0,US,1665318075,94,19.7297,-155.0900,29.04,0.00
227,3382160,Cayenne,0,GF,1665318093,74,4.9333,-52.3333,29.02,2.57
385,128477,Kashan,0,IR,1665318110,13,33.9831,51.4364,29.17,2.93
409,100926,Umluj,0,SA,1665318113,64,25.0213,37.2685,30.03,8.27
416,124193,Meybod,0,IR,1665318114,11,32.2501,54.0166,28.01,3.76
439,6316343,Alta Floresta,0,BR,1665318117,70,-9.8756,-56.0861,29.90,2.06
459,1177446,Gwadar,0,PK,1665318119,38,25.1216,62.3254,30.14,7.01
473,912628,Kasempa,0,ZM,1665318121,13,-13.4584,25.8338,32.57,3.97
520,2212775,Sabha,0,LY,1665318128,24,27.0377,14.4283,28.41,1.78


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Making a copy of the Data Frame in a variable named `hotel_df`
hotel_df = vacation_df.copy()
hotel_df["Hotel Name"] = ""

# Set Google Maps base URL
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Seting parameters to use Google Maps API: look up hotel names within 5000 meters of the city's coordinates
params = {"location" : "",
          "type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

# Iterate all the cities selected as potential holidays
for index, row in hotel_df.iterrows():
    # Updating the parameters to consider the city's coordinates (latitude and longitude)
    params["location"] = f"{row['Lat']},{row['Lng']}"

    # API request and extracting results
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    # Checking if there is a valid response for the hotel name requested in the API call
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        print(f"Retrieving Results for Index {index}: {row['City']} - Hotel found: {results[0]['name']}")
    except:
        hotel_df.loc[index, "Hotel Name"] = ""
        print(f"Retrieving Results for Index {index}: {row['City']} - Hotel NOT found")
hotel_df

Retrieving Results for Index 94: Hilo - Hotel found: Hilo Hawaiian Hotel
Retrieving Results for Index 227: Cayenne - Hotel found: Grand Hotel Montabo
Retrieving Results for Index 385: Kashan - Hotel found: Amir Kabir Hotel
Retrieving Results for Index 409: Umluj - Hotel found: HP Red Sea Hotel
Retrieving Results for Index 416: Meybod - Hotel found: Meybod Behjat Hotel
Retrieving Results for Index 439: Alta Floresta - Hotel found: Hotel Caleche Park Hotel
Retrieving Results for Index 459: Gwadar - Hotel found: Zaver Pearl Continental Hotel
Retrieving Results for Index 473: Kasempa - Hotel found: Comfort Stay Lodge
Retrieving Results for Index 520: Sabha - Hotel found: فندق المواجيد


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
94,5855927,Hilo,0,US,1665318075,94,19.7297,-155.0900,29.04,0.00,Hilo Hawaiian Hotel
227,3382160,Cayenne,0,GF,1665318093,74,4.9333,-52.3333,29.02,2.57,Grand Hotel Montabo
385,128477,Kashan,0,IR,1665318110,13,33.9831,51.4364,29.17,2.93,Amir Kabir Hotel
409,100926,Umluj,0,SA,1665318113,64,25.0213,37.2685,30.03,8.27,HP Red Sea Hotel
416,124193,Meybod,0,IR,1665318114,11,32.2501,54.0166,28.01,3.76,Meybod Behjat Hotel
439,6316343,Alta Floresta,0,BR,1665318117,70,-9.8756,-56.0861,29.90,2.06,Hotel Caleche Park Hotel
459,1177446,Gwadar,0,PK,1665318119,38,25.1216,62.3254,30.14,7.01,Zaver Pearl Continental Hotel
473,912628,Kasempa,0,ZM,1665318121,13,-13.4584,25.8338,32.57,3.97,Comfort Stay Lodge
520,2212775,Sabha,0,LY,1665318128,24,27.0377,14.4283,28.41,1.78,فندق المواجيد


In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [7]:
# Adding the Marker Layer on top of the previous Heatmap
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display the figure
fig

Figure(layout=FigureLayout(height='420px'))